## These are all the cells that were used to create the final training data for XGBoost

## Load data into df_by_ticker


Many columns will be created later and will contain columns such as:`'transaction_date', 'trade_date', 'ticker', 'company_name','owner_name', 'Title', 'transaction_type', 'last_price', 'Qty', 'shares_held', 'Owned', 'Value', 'eff_trans_date', 'Owned_norm', 'prev_prices', 'future_prices', 'SPY_1d_return',  'trade_date_epoch', 'eff_trans_date_epoch', '1mo_return', '3mo_return', '30d_volatility', 'rsi_14', 'macd', 'volume_zscore',  'price_vs_sma50', 'is_during_market_hours', 'day_of_week', 'filing_lad_days', 'title_rank', 'month_end_flag'],`

`'trade_date'`: The date the actual trade was made, tipically a few days prior to the transaction date.

`'transaction_date'`: The date the SEC filing was made.

`'eff_trans_date'`: Effective transaction date. For the purposes of this analysis we are considering before 9:30am (eg: on a Wednesday) as the previous day (Tuesday) all the way up until 9:30am the day prior (Tuesday). transaction made after 9am (on the wednesday) are considered to have been made on that day (Wednesday). This is because the information will not be acted upon until the following day.

`'ticker'`: Stock ticker symbol.

`'company_name'`: It's on the tin.

`'owner_name'`: The name of the person making the transaction (unsure why is plural).

`'Title'`: Title of the person making the trade, CEO, CFO, Dir, 10% stake holder, etc.

`'last_price'`: The last price the stock was bought at during the transaction.

`'Qty'`: Quantity of shares in the trade.

`'shares_held'`: Quantity of shares held before (? maybe after) the trade.

`'Owned'`: Is the percentage increase of owner ship in the company. A new Owner is marked as 'New' in the CSV and changed to 1000000000 to be converted to Owned_norm.

`'Owned_norm'`: Created inorder to deal with the 'New' tag. Owned is normalised from 0 to 1. 0 => they already owned lots, posision has not meaningfully changed. ~0.5 => A 100% increase in shares owned. 1 => they are new to owning this stock.

`'prev_prices'`: The previous 90 days of stock prices leading up to the transaction date (including the transaction date).

`'future_prices'`: The following 30 days of stock prices following the transaction date/


## Initial cleaning

In [ ]:
df = pd.read_csv('2015_2025_data/insider_trades.csv', parse_dates=['transaction_date', 'trade_date'])
df = df.dropna()

for i in ['last_price', 'Qty', 'shares_held', 'Owned', 'Value']:
    df[i] = (
        df[i]
        .astype(str)
        .str.replace('New', '1000000000', regex=False)
        .str.replace('[\$,\+\-\%>]', '', regex=True)
        .astype(float)
    )

df['transaction_date'] = pd.to_datetime(df['transaction_date'], format="%d/%m/%Y %H:%M")
df['trade_date'] = pd.to_datetime(df['trade_date'])

MARKET_OPEN = time(9, 30)
def adjust_transaction_date(dt):
    dt = pd.to_datetime(dt)
    if dt.time() < MARKET_OPEN:
        return (dt - pd.Timedelta(days=1)).date()
    return dt.date()

df['eff_trans_date'] = df['transaction_date'].apply(adjust_transaction_date)

df['Owned_norm'] = 1 - np.exp(-df['Owned']*np.log(2)/100)

df = df.sort_values(by='transaction_date')



<>:9: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
<>:9: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
C:\Users\azzag\AppData\Local\Temp\ipykernel_29156\3460729856.py:9: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
  .str.replace('[\$,\+\-\%>]', '', regex=True)


In [ ]:
df_by_ticker = df.copy()
print(df_by_ticker.shape)


(146307, 14)


## Loading ticker prices into `prices`

`prices` is a pd serise indexed by the ticker names, where each element is pd df with 5 cols, open, high, low, close, volume, with those prices from 2015 to 2025 (or later). 




In [ ]:
with open('saved_vars/Formatted_ticker_data_2015_2025.pkl', 'rb') as f:  # 'rb' = read binary
    prices = pickle.load(f)



## Putting 90days of prices into df_by_ticker

Currently `df_by_ticker` has no price data, and prices has price data from 2015-25, we will select just the 90 days leading up to the trasaction date.

In [ ]:
#Note, for all 150k trades this took 19.2, 22.8 seconds

# Slice per row from this cached dataset
def get_ticker_data_from_cache(trade):
    ticker_prices = prices[trade['ticker']]
    
    prev_price = ticker_prices.loc[trade['eff_trans_date'] - timedelta(days=100):trade['eff_trans_date']]
    #prev_price.columns = prev_price.columns.droplevel(1)

    future_price = ticker_prices.loc[trade['eff_trans_date']+ timedelta(days=1) : trade['eff_trans_date'] + timedelta(days=35)]
    #future_price.columns = future_price.columns.droplevel(1)

    row = {
        'ticker': trade['ticker'],
        'prev_prices': prev_price,
        'future_prices' : future_price
    }
    return row

df_ticker_data = df_by_ticker.apply(get_ticker_data_from_cache, axis=1, result_type='expand')

df_by_ticker['prev_prices'] = df_ticker_data['prev_prices']
df_by_ticker['future_prices'] = df_ticker_data['future_prices']


In [ ]:
print(df_by_ticker.iloc[100])
print(df_by_ticker.iloc[100]['prev_prices'])


transaction_date                                        2015-01-07 17:02:58
trade_date                                              2014-12-31 00:00:00
ticker                                                                   ED
company_name                                        Consolidated Edison Inc
owner_name                                                   Cawley Timothy
Title                                                       Pres, CEO, O, R
transaction_type                                               P - Purchase
last_price                                                            64.69
Qty                                                                    46.0
shares_held                                                          1163.0
Owned                                                                   4.0
Value                                                                2957.0
eff_trans_date                                          2015-01-07 00:00:00
Owned_norm  

## Creating features

#### Download SPY history

In [ ]:
spy_history = yf.Ticker('SPY').history(start='2014-01-01', end='2025-01-01')[['Open', 'High', 'Low', 'Close', 'Volume']]

#spy history deleted in next cell


In [ ]:
#Note: this took 3.2 seconds to run for 5000 trades.
#For all 150k trades it took: 6m 30s, 6m 5s

def create_features(trade):
    """
    ticker_data: DataFrame with OHLCV for one stock over 3 months
    Returns: Single row of engineered features:
        '1mo_return', '3mo_return', '30d_volatility', 'rsi_14', 'macd',
        'volume_zscore', 'price_vs_sma50'.
    """
    ticker_data = trade['prev_prices']

    features = {}

    features['trade_date_epoch'] = pd.to_datetime(trade['trade_date']).timestamp() # seconds since epoch
    features['eff_trans_date_epoch'] = pd.to_datetime(trade['eff_trans_date']).timestamp() 

    
    # Price returns
    features['1mo_return'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].iloc[-22] - 1
    features['3mo_return'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].iloc[-63] - 1
    
    # Volatility, use second line to silence the warning.
    features['30d_volatility'] = ticker_data['Close'].pct_change(fill_method=None).std() * np.sqrt(252)
    

    
    # Momentum indicators
    # RSI 14
    rsi = RSIIndicator(close=ticker_data['Close'], window=14).rsi()  # series
    features['rsi_14'] = rsi.iloc[-1]
    # MACD line
    macd = MACD(close=ticker_data['Close'])
    features['macd'] = macd.macd().iloc[-1]  # MACD line
    
    # Volume
    features['volume_zscore'] = (
        (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean()) 
        / ticker_data['Volume'].std()
    )
    
    # Trend relationships
    features['price_vs_sma50'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].rolling(50).mean().iloc[-1]


    # Day of the week the trade was made on.
    features['day_sin'] = np.sin(2 * np.pi * trade['trade_date'].dayofweek / 7)
    features['day_cos'] = np.cos(2 * np.pi * trade['trade_date'].dayofweek / 7)

    # Number of days between making the trade and filing with the SEC
    features['filing_lad_days'] = (trade['transaction_date']-trade['trade_date']).days


        
    return pd.Series(features)

features = df_by_ticker.apply(lambda trade: create_features(trade), axis=1)



titles = df_by_ticker['Title'].str.join('')
df_by_ticker['title_rank'] = np.select(
    [titles.str.contains('CEO'), titles.str.contains('C'), titles.str.contains('Dir')],
    [4, 3, 2],
    default=1
)

# Looks at the last valid trading date on or before the transaction date.

df_by_ticker['eff_trans_date'] = pd.to_datetime(df_by_ticker['eff_trans_date']).dt.tz_localize(None)
spy_returns = (spy_history['Close'] / spy_history['Open'] - 1).rename('SPY_1d_return')
spy_returns.index = spy_returns.index.tz_localize(None)
df_by_ticker = df_by_ticker.merge(spy_returns, left_on='eff_trans_date', right_index=True, how='left')

# Bool on whether the transaction was made within trading hours.
df_by_ticker['is_during_market_hours'] = (
    df_by_ticker['transaction_date'].dt.time.between(time(9,30), time(16))
)

df_by_ticker[features.columns] = features
#df_by_ticker['month_end_flag'] = flag_month_end(df_by_ticker['trade_date'])
df_by_ticker['day_sin'] = np.sin(2 * np.pi * df_by_ticker['day_of_week'] / 7)
df_by_ticker['day_cos'] = np.cos(2 * np.pi * df_by_ticker['day_of_week'] / 7)
month_end = df_by_ticker['trade_date'] + BMonthEnd(0)
df_by_ticker['month_end_flag'] = ((month_end - df_by_ticker['trade_date']).dt.days < 3).astype(int)
#df_by_ticker = df_by_ticker.dropna()


del spy_history


C:\Users\azzag\AppData\Local\Temp\ipykernel_29156\130387068.py:38: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
C:\Users\azzag\AppData\Local\Temp\ipykernel_29156\130387068.py:38: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
C:\Users\azzag\AppData\Local\Temp\ipykernel_29156\130387068.py:38: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
C:\Users\azzag\AppData\Local\Temp\ipykernel_29156\130387068.py:38: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
C:\Users\azzag\AppData\Local\Temp\ipykernel_29156\130387068.py:38: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
C:\Users\azzag\AppData\Local\Temp\ipyker

### New create new features, untested, added ATR, volume, market cap

In [ ]:
#Note: this took 3.2 seconds to run for 5000 trades.
#For all 150k trades it took: 6m 30s, 6m 5s

def create_features(trade):
    """
    ticker_data: DataFrame with OHLCV for one stock over 3 months
    Returns: Single row of engineered features:
        '1mo_return', '3mo_return', '30d_volatility', 'rsi_14', 'macd',
        'volume_zscore', 'price_vs_sma50'.
    """
    ticker_data = trade['prev_prices']

    features = {}

    features['trade_date_epoch'] = pd.to_datetime(trade['trade_date']).timestamp() # seconds since epoch
    features['eff_trans_date_epoch'] = pd.to_datetime(trade['eff_trans_date']).timestamp() 

    
    # Price returns
    features['1mo_return'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].iloc[-22] - 1
    features['3mo_return'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].iloc[-63] - 1
    
    # Volatility, use second line to silence the warning.
    features['30d_volatility'] = ticker_data['Close'].pct_change(fill_method=None).std() * np.sqrt(252)
    
    #ATR (Average True Range, 14-day)
    high_low = ticker_data['High'] - ticker_data['Low']
    high_close = np.abs(ticker_data['High'] - ticker_data['Close'].shift())
    low_close = np.abs(ticker_data['Low'] - ticker_data['Close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    features['ATR_14'] = tr.rolling(14).mean().iloc[-1]
    
    # Momentum indicators
    # RSI 14
    rsi = RSIIndicator(close=ticker_data['Close'], window=14).rsi()  # series
    features['rsi_14'] = rsi.iloc[-1]
    # MACD line
    macd = MACD(close=ticker_data['Close'])
    features['macd'] = macd.macd().iloc[-1]  # MACD line
    
    # Volume indicators
    features['volume_zscore'] = (
        (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean()) 
        / ticker_data['Volume'].std()
    )
    vol_20 = ticker_data['Volume'].rolling(20).mean().iloc[-1]
    features['Vol_Ratio_20'] = ticker_data['Volume'].iloc[-1] / vol_20 if vol_20 and not np.isnan(vol_20) else np.nan

    
    # Trend relationships
    features['price_vs_sma50'] = ticker_data['Close'].iloc[-1] / ticker_data['Close'].rolling(50).mean().iloc[-1]


    # Day of the week the trade was made on.
    features['day_sin'] = np.sin(2 * np.pi * trade['trade_date'].dayofweek / 7)
    features['day_cos'] = np.cos(2 * np.pi * trade['trade_date'].dayofweek / 7)

    # Number of days between making the trade and filing with the SEC
    features['filing_lad_days'] = (trade['transaction_date']-trade['trade_date']).days


    # --- Market Cap (if shares outstanding provided) ---
    # Requires `trade['shares_outstanding']` or add via merge before apply()
    if 'shares_outstanding' in trade and pd.notnull(trade['shares_outstanding']):
        features['MarketCap'] = ticker_data['Close'].iloc[-1] * trade['shares_outstanding']
    elif 'MarketCap' in ticker_data.columns:
        features['MarketCap'] = ticker_data['MarketCap'].iloc[-1]
    else:
        features['MarketCap'] = np.nan


        
    return pd.Series(features)

features = df_by_ticker.apply(lambda trade: create_features(trade), axis=1)



titles = df_by_ticker['Title'].str.join('')
df_by_ticker['title_rank'] = np.select(
    [titles.str.contains('CEO'), titles.str.contains('C'), titles.str.contains('Dir')],
    [4, 3, 2],
    default=1
)

# Looks at the last valid trading date on or before the transaction date.

df_by_ticker['eff_trans_date'] = pd.to_datetime(df_by_ticker['eff_trans_date']).dt.tz_localize(None)
spy_returns = (spy_history['Close'] / spy_history['Open'] - 1).rename('SPY_1d_return')
spy_returns.index = spy_returns.index.tz_localize(None)
df_by_ticker = df_by_ticker.merge(spy_returns, left_on='eff_trans_date', right_index=True, how='left')

# Bool on whether the transaction was made within trading hours.
df_by_ticker['is_during_market_hours'] = (
    df_by_ticker['transaction_date'].dt.time.between(time(9,30), time(16))
)

df_by_ticker[features.columns] = features
#df_by_ticker['month_end_flag'] = flag_month_end(df_by_ticker['trade_date'])
df_by_ticker['day_sin'] = np.sin(2 * np.pi * df_by_ticker['day_of_week'] / 7)
df_by_ticker['day_cos'] = np.cos(2 * np.pi * df_by_ticker['day_of_week'] / 7)
month_end = df_by_ticker['trade_date'] + BMonthEnd(0)
df_by_ticker['month_end_flag'] = ((month_end - df_by_ticker['trade_date']).dt.days < 3).astype(int)
#df_by_ticker = df_by_ticker.dropna()


del spy_history


In [ ]:
def count_same_day_trades(df):
    """
    Adds a column 'same_day_trade_count' representing
    how many trades occurred for the same company on the same eff_trans_date.
    """
    counts = (
        df.groupby(['ticker', 'eff_trans_date'])
        .size()
        .rename('same_day_trade_count')
    )
    return df.merge(counts, on=['ticker', 'eff_trans_date'], how='left')

df_by_ticker = count_same_day_trades(df_by_ticker)

In [ ]:
# Precompute the relevant indices
open_prices = df_by_ticker['future_prices'].apply(lambda df: df['Open'].iloc[0])
print('Retreived day 1 open prices')
next_day_close = df_by_ticker['future_prices'].apply(lambda df: df['Close'].iloc[0])
print('Retreived day 1 close prices')
one_week_close = df_by_ticker['future_prices'].apply(lambda df: df['Close'].iloc[5])
print('Retreived week 1 close prices')
one_month_close = df_by_ticker['future_prices'].apply(lambda df: df['Close'].iloc[20])
print('Retreived month 1 close prices')

# Define thresholds
thresholds = [1, 1.02, 1.05]

# Compute targets in a vectorised way
targets_dict = {}
for t in thresholds:
    targets_dict[f'next_day_{int((t-1)*100) if t != 1 else ""}%'] = next_day_close / open_prices > t
    targets_dict[f'one_week_{int((t-1)*100) if t != 1 else ""}%'] = one_week_close / open_prices > t
    targets_dict[f'one_month_{int((t-1)*100) if t != 1 else ""}%'] = one_month_close / open_prices > t

targets = pd.DataFrame(targets_dict)

del open_prices, next_day_close, one_week_close, one_month_close